In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests 
import time
import csv
import re
import os
import csv
from unidecode import unidecode
from selenium import webdriver

In [32]:
def get_user_following(url):
    
    break_levels = 0
    
    global following
    following = []
    
    #Number of Pages to consider for each User --> here is 99 pages in Maximum
    #Each page in virgool contains 40 users data
    for i in range(1,100):
        
        user_following_link = url+"/following/{}".format(i)
        driver.get(user_following_link)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        #Sometimes the page is empty or an error page pops-up
        #In that case we need to stop and go to another User
        if not soup.find_all('div',{'class':'listview-item author'}):
            break
        
        #Goes through 40 users that the source user follows and picks their information
        for user in soup.find_all('div',{'class':'listview-item author'}):
            user_identity = user.find('a',{'class':'item--title'}, href=True)
            user_desc = user.find('div',{'class': 'item--descreption'})
            
            #If the information is repetitive, it shows that the bot is at the last page and should break
            if ([url[20:],user_identity['href'][20:],user_identity.text,user_identity['href'],user_desc.text[1:-1]]) in following:
                break_levels = 1
                break
            else:
                following.append([url[20:],user_identity['href'][20:],user_identity.text,user_identity['href'],user_desc.text[1:-1]])

        #time.sleep(2)
        if break_levels == 1:
            break

        
    return following

In [30]:
#A simple function to add each users data to a CSV file
def write_to_csv(file_path, header, data):
    file_exists = os.path.isfile(file_path)
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        if file_exists != True:
            writer.writerow(i for i in header)
        writer.writerow(data)

In [ ]:
#Get User Followings
output_dir = 'Desktop'
user_following_csv_file_path = os.path.join(output_dir, 'following.csv')


driver = webdriver.Chrome('chromedriver')

#How many hops to go?
hops = 5

#set a user to start crawling from
nodes_list = {0:['https://virgool.io/@saeed.choobani']}

#create a remove set to prevent crawling replicated users
removeset = set()

#In each hope we create a dictionary of the next hop's links
for k in range(hops):
    
    nodes_list.update({k+1:[]})
    
    for node in nodes_list[k]:

        
        get_user_following(node)

        
        for j in following:
            header = ['source', 'target', 'target_name', 'target_url', 'target_desc']
            write_to_csv(user_following_csv_file_path, header, j)
        
        
        following = np.array(following)
        try:
            for a in following[:,3].tolist():
                nodes_list[k+1].append(a)
        except:
            continue
            
    
    removeset.update(nodes_list[k])
    nodes_list[k+1] = [x for x in nodes_list[k+1] if x not in removeset]

driver.quit()

In [147]:
#and Extra function in case you want to gather data from user page
def get_user_info(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for i in soup.find_all('div', {'class':'follow-count js-follow-section-count'}):
        follow_count = i.find_all('span')

    following_count = int(unidecode(u"{}".format(follow_count[1].text)))
    follower_count = int(unidecode(u"{}".format(follow_count[0].text)))
    user_identity = soup.find('a', {'class':'module--name'}, href=True)
    user_fullname = user_identity.text[1:-1]
    user_url = user_identity['href']
    username = user_identity['href'][20:]
    user_desc = soup.find('p', {'class':'module--bio js--linkify'}).text


    return (username,user_fullname,user_desc, following_count, follower_count)
    